In [ ]:
import numpy as np
import os
import cv2
from sklearn.metrics.pairwise import cosine_similarity
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.models import Model
import matplotlib.pyplot as plt

# Load pre-trained VGG16 model + higher level layers
base_model = VGG16(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)

# Directory containing images
IMAGE_DIR = 'path_to_images'  # Replace with your directory

# Function to preprocess images and extract features using VGG16
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))  # VGG16 requires 224x224 input
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)

    features = model.predict(img_data)
    return features.flatten()

# Function to load images and compute feature vectors
def load_images_and_extract_features(image_dir, model):
    feature_list = []
    image_paths = []
    for img_file in os.listdir(image_dir):
        if img_file.endswith(('.jpg', '.png', '.jpeg')):
            img_path = os.path.join(image_dir, img_file)
            features = extract_features(img_path, model)
            feature_list.append(features)
            image_paths.append(img_path)
    return np.array(feature_list), image_paths

# Load image features from the directory
features, img_paths = load_images_and_extract_features(IMAGE_DIR, model)

# Function to search for visually similar images using cosine similarity
def search_similar_images(query_img_path, feature_list, image_paths, model, top_n=5):
    query_features = extract_features(query_img_path, model)
    similarities = cosine_similarity([query_features], feature_list)[0]
    indices = np.argsort(similarities)[::-1][:top_n]

    similar_images = [(image_paths[i], similarities[i]) for i in indices]
    return similar_images

# Displaying similar images
def display_similar_images(similar_images):
    plt.figure(figsize=(10, 5))
    for i, (img_path, similarity) in enumerate(similar_images):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.subplot(1, len(similar_images), i+1)
        plt.imshow(img)
        plt.title(f"Similarity: {similarity:.2f}")
        plt.axis('off')
    plt.show()

# Test the CBIR system with a query image
query_image_path = 'path_to_query_image'  # Replace with your query image path
similar_images = search_similar_images(query_image_path, features, img_paths, model)

# Display the similar images
display_similar_images(similar_images)